
# Рейтинг заявок — очистка, об'єднання та агрегація (Pandas)

**Джерело даних:** Google Sheets (публічні лінки).  
**Формат здачі:** цей ноутбук містить усі кроки 1–5 і виводи таблиць, як вимагає ТЗ.


In [1]:

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 200)

apps_url = "https://docs.google.com/spreadsheets/d/1CNGHjNcJgZPYO-eR1i-1-GQ9fVqYkOnHcwpiZhA4bvg/export?format=csv"
inds_url = "https://docs.google.com/spreadsheets/d/1goV7_lVeRUVrX9pq1Pa7HJlYc8_i20-kizVLSr9p7gM/export?format=csv"

applications = pd.read_csv(apps_url)
industries   = pd.read_csv(inds_url)

print("applications shape:", applications.shape)
print("industries shape:", industries.shape)

display(applications.head())
display(industries.head())


applications shape: (13315, 10)
industries shape: (37, 2)


,Applied at,Amount,Age,Gender,Industry,Marital status,External Rating,Education level,Location,applicant_id
0,11.30.2022 10:26:37,12000.0,29,Чоловік,Blockchain,Other,8.0,"Вища (бакалавр, спеціаліст, магістр)",Івано-Франківськ чи область,99e7b0dc6cc05dd334d8f38dc26ce9b3
1,11.30.2022 10:26:39,NaN,36,Чоловік,Public services / Government,Single,3.0,"Вища (бакалавр, спеціаліст, магістр)",NaN,63dfcf8e6904186650d6814279fbe42f
2,11.30.2022 10:26:58,7500.0,34,Чоловік,Adtech / Advertising,Single,4.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,10dbafaeb46c09e96b6987c03bbb3498
3,11.30.2022 10:27:31,1500.0,23,Жінка,Telecom,Single,0.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,5847ac62cc9eac5e323c2517dcc91ad1
4,11.30.2022 10:27:34,8400.0,33,Жінка,Automotive,Single,6.0,"Вища (бакалавр, спеціаліст, магістр)",Житомир чи область,5d21f3795b50de8e8f8f8d5f48b754f3


,Industry,Score
0,Blockchain,0
1,Public services / Government,20
2,Adtech / Advertising,10
3,Telecom,15
4,Automotive,15



## Крок 1. Очищення даних (applications)
- Прибрати дублі за `applicant_id`
- Заповнити NaN у **External Rating** нулями
- Заповнити NaN у **Education level** текстом **"Середня"**


In [2]:

apps_clean = applications.drop_duplicates(subset="applicant_id", keep="first").copy()

apps_clean["External Rating"] = apps_clean["External Rating"].fillna(0)
apps_clean["Education level"] = apps_clean["Education level"].fillna("Середня")

print({
    "rows_before": len(applications),
    "rows_after": len(apps_clean),
    "duplicates_removed": len(applications) - len(apps_clean),
    "missing_external_rating": int(apps_clean["External Rating"].isna().sum()),
    "missing_education": int(apps_clean["Education level"].isna().sum()),
})

display(apps_clean.head())


{'rows_before': 13315, 'rows_after': 13278, 'duplicates_removed': 37, 'missing_external_rating': 0, 'missing_education': 0}


,Applied at,Amount,Age,Gender,Industry,Marital status,External Rating,Education level,Location,applicant_id
0,11.30.2022 10:26:37,12000.0,29,Чоловік,Blockchain,Other,8.0,"Вища (бакалавр, спеціаліст, магістр)",Івано-Франківськ чи область,99e7b0dc6cc05dd334d8f38dc26ce9b3
1,11.30.2022 10:26:39,NaN,36,Чоловік,Public services / Government,Single,3.0,"Вища (бакалавр, спеціаліст, магістр)",NaN,63dfcf8e6904186650d6814279fbe42f
2,11.30.2022 10:26:58,7500.0,34,Чоловік,Adtech / Advertising,Single,4.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,10dbafaeb46c09e96b6987c03bbb3498
3,11.30.2022 10:27:31,1500.0,23,Жінка,Telecom,Single,0.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,5847ac62cc9eac5e323c2517dcc91ad1
4,11.30.2022 10:27:34,8400.0,33,Жінка,Automotive,Single,6.0,"Вища (бакалавр, спеціаліст, магістр)",Житомир чи область,5d21f3795b50de8e8f8f8d5f48b754f3



## Крок 2. Додати рейтинги індустрій (join з `industries.csv`)


In [3]:

apps_merged = apps_clean.merge(
    industries,
    on="Industry",
    how="left"
)

print("Після об'єднання:", apps_merged.shape)
print("Пропуски у Score:", int(apps_merged["Score"].isna().sum()))
display(apps_merged.head())


Після об'єднання: (13278, 11)
Пропуски у Score: 0


,Applied at,Amount,Age,Gender,Industry,Marital status,External Rating,Education level,Location,applicant_id,Score
0,11.30.2022 10:26:37,12000.0,29,Чоловік,Blockchain,Other,8.0,"Вища (бакалавр, спеціаліст, магістр)",Івано-Франківськ чи область,99e7b0dc6cc05dd334d8f38dc26ce9b3,0
1,11.30.2022 10:26:39,NaN,36,Чоловік,Public services / Government,Single,3.0,"Вища (бакалавр, спеціаліст, магістр)",NaN,63dfcf8e6904186650d6814279fbe42f,20
2,11.30.2022 10:26:58,7500.0,34,Чоловік,Adtech / Advertising,Single,4.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,10dbafaeb46c09e96b6987c03bbb3498,10
3,11.30.2022 10:27:31,1500.0,23,Жінка,Telecom,Single,0.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,5847ac62cc9eac5e323c2517dcc91ad1,15
4,11.30.2022 10:27:34,8400.0,33,Жінка,Automotive,Single,6.0,"Вища (бакалавр, спеціаліст, магістр)",Житомир чи область,5d21f3795b50de8e8f8f8d5f48b754f3,15



## Крок 3. Розрахунок рейтингу заявки

**Умови:**  
- Рейтинг ∈ [0; 100]  
- Рейтинг — сума балів за 6 критеріями  
- Якщо `Amount` відсутній **або** `External Rating == 0` → рейтинг = 0

**Складові:**  
1) Вік 35–55 → +20  
2) Подано не у вихідні → +20  
3) Одружений → +20 (значення `Одружений`)  
4) Локація Київ або область → +10 (шукаємо підрядок `київ`)  
5) Score індустрії → +0..20 (з таблиці `industries`)  
6) `External Rating` ≥ 7 → +20, `External Rating` ≤ 2 → −20


In [4]:

df = apps_merged.copy()

# Дата заявки для визначення дня тижня
df["Applied at"] = pd.to_datetime(df["Applied at"], errors="coerce")

# Умови валідності для рейтингу
df["valid"] = (~df["Amount"].isna()) & (df["External Rating"] > 0)

# 1) Вік 35–55
age_points = ((df["Age"] >= 35) & (df["Age"] <= 55)) * 20

# 2) Не вихідні (пн-пт → 0..4)
weekday_points = (df["Applied at"].dt.weekday < 5) * 20

# 3) Одружений
married_points = df["Marital status"].astype(str).str.lower().eq("одружений") * 20

# 4) Київ або область (шукаємо 'київ')
location_points = df["Location"].astype(str).str.contains("київ", case=False, na=False) * 10

# 5) Бал індустрії
industry_points = df["Score"].fillna(0)

# 6) External Rating
external_points = (df["External Rating"] >= 7) * 20 - (df["External Rating"] <= 2) * 20

# Підсумок
df["Rating_raw"] = age_points + weekday_points + married_points + location_points + industry_points + external_points

# Обнулення за правилом і обрізання до [0,100]
df["Rating"] = (df["Rating_raw"] * df["valid"]).clip(lower=0, upper=100)

print("Контроль діапазону рейтингу: min =", float(df["Rating"].min()), ", max =", float(df["Rating"].max()))
display(df[["applicant_id","Age","Applied at","Marital status","Location","Score","External Rating","Amount","Rating_raw","valid","Rating"]].head(10))


Контроль діапазону рейтингу: min = 0.0 , max = 90.0


,applicant_id,Age,Applied at,Marital status,Location,Score,External Rating,Amount,Rating_raw,valid,Rating
0,99e7b0dc6cc05dd334d8f38dc26ce9b3,29,2022-11-30 10:26:37,Other,Івано-Франківськ чи область,0,8.0,12000.0,40,True,40
1,63dfcf8e6904186650d6814279fbe42f,36,2022-11-30 10:26:39,Single,NaN,20,3.0,NaN,60,False,0
2,10dbafaeb46c09e96b6987c03bbb3498,34,2022-11-30 10:26:58,Single,Київ чи область,10,4.0,7500.0,40,True,40
3,5847ac62cc9eac5e323c2517dcc91ad1,23,2022-11-30 10:27:31,Single,Львів чи область,15,0.0,1500.0,15,False,0
4,5d21f3795b50de8e8f8f8d5f48b754f3,33,2022-11-30 10:27:34,Single,Житомир чи область,15,6.0,8400.0,35,True,35
5,f720bf9c5c4c3e10a8568c1699847696,31,2022-11-30 10:27:38,Single,Київ чи область,15,8.0,16500.0,65,True,65
6,0aaf59fb3ef90f50ccd4800312e5c271,30,2022-11-30 10:27:42,Married,Київ чи область,5,1.0,4200.0,15,True,15
7,aa54d6e672e4233febadbafdfd048327,24,2022-11-30 10:27:44,Married,Львів чи область,15,1.0,3600.0,15,True,15
8,2eed2883cd1673b21b2ce89d1115c245,27,2022-11-30 10:28:00,Married,Львів чи область,15,1.0,1950.0,15,True,15
9,9c7a640dccb37d9f2164bac1deb0edfa,17,2022-11-30 10:28:00,Married,NaN,10,2.0,NaN,10,False,0



## Крок 4. Залишити лише заявки з рейтингом > 0 (прийняті)


In [5]:

accepted = df[df["Rating"] > 0].copy()
print("Кількість прийнятих заявок:", len(accepted))
display(accepted.head(10))


Кількість прийнятих заявок: 11188


,Applied at,Amount,Age,Gender,Industry,Marital status,External Rating,Education level,Location,applicant_id,Score,valid,Rating_raw,Rating
0,2022-11-30 10:26:37,12000.0,29,Чоловік,Blockchain,Other,8.0,"Вища (бакалавр, спеціаліст, магістр)",Івано-Франківськ чи область,99e7b0dc6cc05dd334d8f38dc26ce9b3,0,True,40,40
2,2022-11-30 10:26:58,7500.0,34,Чоловік,Adtech / Advertising,Single,4.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,10dbafaeb46c09e96b6987c03bbb3498,10,True,40,40
4,2022-11-30 10:27:34,8400.0,33,Жінка,Automotive,Single,6.0,"Вища (бакалавр, спеціаліст, магістр)",Житомир чи область,5d21f3795b50de8e8f8f8d5f48b754f3,15,True,35,35
5,2022-11-30 10:27:38,16500.0,31,Чоловік,E-commerce,Single,8.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,f720bf9c5c4c3e10a8568c1699847696,15,True,65,65
6,2022-11-30 10:27:42,4200.0,30,Чоловік,Media,Married,1.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,0aaf59fb3ef90f50ccd4800312e5c271,5,True,15,15
7,2022-11-30 10:27:44,3600.0,24,Чоловік,E-commerce,Married,1.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,aa54d6e672e4233febadbafdfd048327,15,True,15,15
8,2022-11-30 10:28:00,1950.0,27,Чоловік,Automotive,Married,1.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,2eed2883cd1673b21b2ce89d1115c245,15,True,15,15
10,2022-11-30 10:28:03,18000.0,25,Чоловік,Dating,Single,5.0,"Вища (бакалавр, спеціаліст, магістр)",Дніпро чи область,f8138219d5a95649cc85bdabeb3732ca,5,True,25,25
12,2022-11-30 10:28:27,2550.0,20,Чоловік,Legal,Married,1.0,Ще студент вишу,Івано-Франківськ чи область,0c792f52f2bf30cdfee53b24b912678a,20,True,20,20
13,2022-11-30 10:28:28,18000.0,33,Чоловік,Big Data,Single,10.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,28165b1a85ee2e48782cada5f2a660b4,20,True,70,70



## Крок 5. Групування за тижнем подачі і середній рейтинг прийнятих заявок


In [7]:

# Тиждень від понеділка (як дата початку тижня)
accepted = accepted.dropna(subset=["Applied at"]).copy()
accepted["week"] = accepted["Applied at"].dt.to_period("W").dt.start_time

weekly_rating = (
    accepted.groupby("week")["Rating"]
    .mean()
    .reset_index()
    .rename(columns={"Rating": "Avg Rating"})
)

print("Середній рейтинг прийнятих заявок по тижнях:")
display(weekly_rating.head(20))



Середній рейтинг прийнятих заявок по тижнях:


,week,Avg Rating
0,2022-11-28,42.300843
1,2022-12-12,38.428812
2,2022-12-19,36.894380
3,2022-12-26,40.368231
